In [2]:
import warnings 
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import os
import random
import numpy as np

import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
from datetime import datetime, timezone, timedelta
from sklearn.model_selection import StratifiedGroupKFold

from wandb.lightgbm import wandb_callback, log_summary

In [17]:
#wandb_callback 수정 
from typing import TYPE_CHECKING, Callable
import wandb
from wandb.sdk.lib import telemetry as wb_telemetry

MINIMIZE_METRICS = [
    "l1",
    "l2",
    "rmse",
    "mape",
    "huber",
    "fair",
    "poisson",
    "gamma",
    "binary_logloss",
]

MAXIMIZE_METRICS = ["map", "auc", "average_precision"]

def _define_metric(data: str, metric_name: str) -> None:
    
    """Capture model performance at the best step.

    instead of the last step, of training in your `wandb.summary`
    """
    if "loss" in str.lower(metric_name):
        wandb.define_metric(f"{data}_{metric_name}", summary="min")
    elif str.lower(metric_name) in MINIMIZE_METRICS:
        wandb.define_metric(f"{data}_{metric_name}", summary="min")
    elif str.lower(metric_name) in MAXIMIZE_METRICS:
        wandb.define_metric(f"{data}_{metric_name}", summary="max")
        
def wandb_callback(log_params: bool = True, define_metric: bool = True) -> Callable:
    """Automatically integrates LightGBM with wandb.

    Arguments:
        log_params: (boolean) if True (default) logs params passed to lightgbm.train as W&B config
        define_metric: (boolean) if True (default) capture model performance at the best step, instead of the last step, of training in your `wandb.summary`

    Passing `wandb_callback` to LightGBM will:
      - log params passed to lightgbm.train as W&B config (default).
      - log evaluation metrics collected by LightGBM, such as rmse, accuracy etc to Weights & Biases
      - Capture the best metric in `wandb.summary` when `define_metric=True` (default).

    Use `log_summary` as an extension of this callback.

    Example:
        ```python
        params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            .
        }
        gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=10,
                        valid_sets=lgb_eval,
                        valid_names=('validation'),
                        callbacks=[wandb_callback()])
        ```
    """
    log_params_list: "List[bool]" = [log_params]
    define_metric_list: "List[bool]" = [define_metric]

    def _init(env: "CallbackEnv") -> None:
        with wb_telemetry.context() as tel:
            tel.feature.lightgbm_wandb_callback = True

        wandb.config.update(env.params)
        log_params_list[0] = False

        if define_metric_list[0]:
            for i in range(len(env.evaluation_result_list)):
                data_type = env.evaluation_result_list[i][0]
                metric_name = env.evaluation_result_list[i][1]
                _define_metric(data_type, metric_name)

    def _callback(env: "CallbackEnv") -> None:
        if log_params_list[0]:
            _init(env)
        # eval_results: "Dict[str, Dict[str, List[Any]]]" = {}
        # recorder = lightgbm.record_evaluation(eval_results)
        # recorder(env)
        eval_results = {x[0]:{x[1:][0]:x[1:][1:]} for x in env.evaluation_result_list}

        for validation_key in eval_results.keys():
            for key in eval_results[validation_key].keys():
                 wandb.log(
                     {validation_key + "_" + key: eval_results[validation_key][key][0]},
                     commit=False,
                 )
        for item in eval_results:
            if len(item) == 4:
                wandb.log({f"{item[0]}_{item[1]}": item[2]}, commit=False)

        # Previous log statements use commit=False. This commits them.
        wandb.log({"iteration": env.iteration}, commit=True)

    return _callback

## Training

In [8]:
#경로에 맞게 수정
X=pd.read_parquet('/data/ephemeral/level2-dkt-recsys-06/data/train_ppd_final_sfcv.parquet')
test=pd.read_parquet('/data/ephemeral/level2-dkt-recsys-06/data/test_ppd_final.parquet')

In [9]:
y=X["answerCode"]
g=X["userID"]

In [10]:
feat=[ 'KnowledgeTag', 
       'user_test_correct_answer', 
       'user_test_total_answer',
       'user_test_acc', 
       'user_tag_correct_answer', 
       'user_tag_total_answer',
       'user_tag_acc', 
       'testid_first', 
       'testid_rest', 
       'itemseq', 
       'item_mean',
       'test_mean', 
       'tag_mean', 
       'item_std', 
       'test_std', 
       'tag_std', 
       'month',
       'hour', 
       'repeat', 
       'elapse', 
       'total_elapse', 
       'encoded_time',
       'user_tag_incorrect', 
       'user_tag_inacc' ]

In [18]:
params={'objective': 'binary', 
        'metric': ['auc'],
        'device':'cpu',
        'num_boost_round':200, 
        'early_stopping_rounds':20
        }

n_fold=5
sfcv=StratifiedGroupKFold(n_splits=n_fold)
oof_auc = np.zeros(n_fold)
oof_acc = np.zeros(n_fold)
test_preds = np.zeros(len(test))

for i , (train_idx, val_idx) in enumerate(sfcv.split(X, y, g)):
    print(f"Fold {i}:")
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_valid = X.iloc[val_idx]
    X_valid = X_valid[X_valid['userID'] != X_valid['userID'].shift(-1)]
    y_valid = X_valid["answerCode"]
    lgb_train = lgb.Dataset(X_train[feat], y_train, categorical_feature=["KnowledgeTag"])
    lgb_valid = lgb.Dataset(X_valid[feat], y_valid, categorical_feature=["KnowledgeTag"])
    wandb.init(project="dkt", config=params)
    wandb.run.name = "fold"+str(i)+"lgbm"
    model = lgb.train(
                    params, 
                    lgb_train,
                    valid_sets=[lgb_train, lgb_valid], 
                    callbacks=[wandb_callback(), lgb.log_evaluation()],
                    categorical_feature=["KnowledgeTag"]
                    )
    #log_summary(model, save_model_checkpoint=True)
    #wandb.finish()
    preds = model.predict(X_valid[feat])
    oof_acc[i] = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    oof_auc[i] = roc_auc_score(y_valid, preds)
    
    test_preds+=model.predict(test)/n_fold
    
    print(f'Fold {i} VALID AUC : {oof_auc[i]} ACC : {oof_acc[i]}\n')

Fold 0:


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /opt/ml/.netrc


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 1322873, number of negative: 697897
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024780 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3897
[LightGBM] [Info] Number of data points in the train set: 2020770, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639490
[LightGBM] [Info] Start training from score 0.639490
[1]	training's auc: 0.810993	valid_1's auc: 0.794067
Training until validation scores don't improve for 20 rounds
[2]	training's auc: 0.814312	valid_1's auc: 0.798142
[3]	training's auc: 0.818308	valid_1's au

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


iteration,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
training_auc,▁▃▃▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
valid_1_auc,▁▃▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇███████████████████
iteration,199


...

iteration,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
training_auc,▁▂▃▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
valid_1_auc,▁▃▄▅▅▆▆▆▇▇▇▇▇▇▇▇████████████████████████
iteration,199


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 1322858, number of negative: 697894
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027382 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3901
[LightGBM] [Info] Number of data points in the train set: 2020752, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654636 -> initscore=0.639483
[LightGBM] [Info] Start training from score 0.639483
[1]	training's auc: 0.811147	valid_1's auc: 0.796528
Training until validation scores don't improve for 20 rounds
[2]	training's auc: 0.814935	valid_1's auc: 0.800836
[3]	training's auc: 0.818605	valid_1's au

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


iteration,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
training_auc,▁▂▃▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
valid_1_auc,▁▂▄▄▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████████████████
iteration,199


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 1322866, number of negative: 697898
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.025203 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3921
[LightGBM] [Info] Number of data points in the train set: 2020764, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654637 -> initscore=0.639483
[LightGBM] [Info] Start training from score 0.639483
[1]	training's auc: 0.811178	valid_1's auc: 0.797588
Training until validation scores don't improve for 20 rounds
[2]	training's auc: 0.814838	valid_1's auc: 0.800416
[3]	training's auc: 0.817051	valid_1's au

In [19]:
np.mean(oof_auc), np.mean(oof_acc)

(0.8337498803325818, 0.7556318614774629)

In [20]:
output_dir = 'output/'
write_path = os.path.join(output_dir, datetime.now(timezone(timedelta(hours=9))).strftime("%Y-%m-%d %H:%M:%S")+" lgbm submission.csv")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
with open(write_path, 'w', encoding='utf8') as w:
    print("writing prediction : {}".format(write_path))
    w.write("id,prediction\n")
    for id, p in enumerate(test_preds):
        w.write('{},{}\n'.format(id,p))

writing prediction : output/2024-01-18 23:16:09 lgbm submission.csv
